# ResNet-34 Architecture Graph Implementation in TensorFlow

In [1]:
import numpy as np
import tensorflow as tf

//anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
//anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Building the Graph

In [2]:
tf.reset_default_graph()

### Inputs, Placeholders, and Constants

In [3]:
NUM_CLASSES = 10
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
x = tf.placeholder(tf.float32, shape=[None,IMAGE_WIDTH,IMAGE_HEIGHT,3]) # represents input 227 x 227 image with 3 color channels (RGB)
y_true = tf.placeholder(tf.float32, shape=[None, 10])
hold_prob = tf.placeholder(tf.float32)
training = tf.placeholder(tf.bool) # Used for batch normalization - a boolean to indicate whether or not we are training

### Helper Functions

In [4]:
def init_weights(shape):  # initializes the weights randomly with a normal distribution
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape): # inditializes the bias term as a constant of 0.1 values
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W, pad=True, strides=[1,1,1,1]): # creates a 2D convolution with or without padding
    if pad:
        return tf.nn.conv2d(x, W, strides=strides, padding='SAME')
    else:
        return tf.nn.conv2d(x, W, strides=strides, padding='VALID')

def max_pool_nbyn(x, name, filter_size=2, stride=2, pad=True):   # creates a max pooling layer
    if pad:
        return tf.nn.max_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='SAME', name=name)
    else:
        return tf.nn.max_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='VALID', name=name)
    
def average_pool_nbyn(x, name, filter_size=2, stride=2, pad=True):   # creates a max pooling layer
    if pad:
        return tf.nn.avg_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='SAME', name=name)
    else:
        return tf.nn.avg_pool(x, ksize=[1, filter_size, filter_size, 1],
                          strides=[1, stride, stride, 1], padding='VALID', name=name)

def global_average_pool(x):
    
    return tf.reshape(tf.reduce_mean(x, [1,2]), [-1, 1, 1, x.get_shape().as_list()[-1]])

def convolutional_layer(input_x, shape, strides=[1,1,1,1]):  # creates the convolutional layer including the weights and biases
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W, strides) + b) # applies a Rectified Linear Unit (ReLU) activation function

def normal_full_layer(input_layer, size):   # creates the fully connected layer
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b  # simple forward propagation using matrix multiplication

def batch_normalization(input_layer, training):  # function for batch normalization
    
    return tf.layers.batch_normalization(input_layer, training=training)

def local_response_normalization(input_layer, radius, alpha, beta, name, bias=1.0): # function for local response normalization
    
     return tf.nn.local_response_normalization(x, depth_radius=radius,
                                              alpha=alpha, beta=beta,
                                              bias=bias, name=name)


def conv(x, filter_height, filter_width, num_filters, stride_y, stride_x, name,
         padding='SAME', groups=1, add_relu=True):
    """Create a convolution layer.
    Adapted from: https://github.com/ethereon/caffe-tensorflow
    """
    # Get number of input channels
    input_channels = int(x.get_shape()[-1])

    # Create lambda function for the convolution
    convolve = lambda i, k: tf.nn.conv2d(i, k,
                                         strides=[1, stride_y, stride_x, 1],
                                         padding=padding)

    with tf.variable_scope(name) as scope:
        # Create tf variables for the weights and biases of the conv layer
        weights = tf.get_variable('weights', shape=[filter_height,
                                                    filter_width,
                                                    input_channels/groups,
                                                    num_filters])
        biases = tf.get_variable('biases', shape=[num_filters])

    if groups == 1:
        conv = convolve(x, weights)

    # In the cases of multiple groups, split inputs & weights and
    else:
        # Split input and weights and convolve them separately
        input_groups = tf.split(axis=3, num_or_size_splits=groups, value=x)
        weight_groups = tf.split(axis=3, num_or_size_splits=groups,
                                 value=weights)
        output_groups = [convolve(i, k) for i, k in zip(input_groups, weight_groups)]

        # Concat the convolved output together again
        conv = tf.concat(axis=3, values=output_groups)

    # Add biases
    bias = tf.reshape(tf.nn.bias_add(conv, biases), tf.shape(conv))

    # Apply relu function
    if add_relu:
        relu = tf.nn.relu(bias, name=scope.name)
        return relu
    else:
        return bias

### Function to Generate Simple Two-Layer Residual Units 
Based on the following diagram from the ResNet paper.
<img src="resnet_block.png" style="width: 400px; height: 350px;"/>

In [5]:
def residual_unit(input_layer, output_channels, name, pad_zero_dim_match=True):
    
    input_channels = input_layer.get_shape().as_list()[-1] # Gets the number of channels in the input
    
    if input_channels * 2 == output_channels:
        increase_dim = True
        stride = 2
    elif input_channels == output_channels:
        increase_dim = False
        stride = 1
    else:
        raise ValueError('Output and input channel do not match in residual blocks!')
    
    if increase_dim is True:
        if pad_zero_dim_match: # If the we are using the zero-padding approach to match the dimensions
            pooled_input = average_pool_nbyn(input_layer, filter_size=2, stride=2, pad=False, name=(name+'_pooled_input'))
            padded_input = tf.pad(pooled_input, [[0, 0], [0, 0], [0, 0], [input_channels // 2,
                                                                     input_channels // 2]])
            conv_1 = conv(input_layer, 3, 3, output_channels, 2, 2, groups=1, name=(name+'_conv_1'), add_relu=False)
            batch_norm_1 = tf.nn.relu(batch_normalization(conv_1, training))
            conv_2 = conv(batch_norm_1, 3, 3, output_channels, 1, 1, groups=1, add_relu=False, name=(name+'_conv_2'))
            
            return tf.nn.relu(batch_normalization(conv_2, training) + padded_input)
        else: # Otherwise the approach for dimension-matching involves 1 x 1 convolutions
            conv_1x1 = conv(input_layer, 1, 1, output_channels, 2, 2, groups=1, name=(name+'_conv_1x1'), add_relu=false)
            conv_1x1_batch_norm = tf.nn.relu(batch_normalization(conv_1x1))
            
            conv_1 = conv(input_layer, 3, 3, output_channels, 2, 2, groups=1, name=(name+'_conv_1'), add_relu=False)
            batch_norm_1 = tf.nn.relu(batch_normalization(conv_1, training))
            conv_2 = conv(batch_norm_1, 3, 3, output_channels, 1, 1, groups=1, add_relu=False, name=(name+'_conv_2'))
            
            return tf.nn.relu(batch_normalization(conv_2, training) + conv_1x1_batch_norm)
        
    else:
        conv_1 = conv(input_layer, 3, 3, output_channels, 1, 1, groups=1, name=(name+'_conv_1'))
        batch_norm_1 = tf.nn.relu(batch_normalization(conv_1, training))
        conv_2 = conv(batch_norm_1, 3, 3, output_channels, 1, 1, groups=1, add_relu=False, name=(name+'_conv_2'))
            
        return tf.nn.relu(batch_normalization(conv_2, training) + input_layer)
        

### CONV 1: 64 7 x 7 filters with stride = 2

In [6]:
conv_1 = tf.nn.relu(batch_normalization(conv(x, 7, 7, 64, 2, 2, groups=1, name='conv_1', add_relu=False), training))
conv_1.shape

TensorShape([Dimension(None), Dimension(112), Dimension(112), Dimension(64)])

### POOL 1: 3 x 3 filter, stride = 2

In [7]:
pool_1 = max_pool_nbyn(conv_1, filter_size=3, stride=2, pad=True, name='pool_1')
pool_1.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 1: Two stacked 64-filter 3 x 3 conv layers

In [8]:
res_unit_1 = residual_unit(pool_1, 64, name='res_unit_1')
res_unit_1.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 2: Two stacked 64-filter 3 x 3 conv layers

In [9]:
res_unit_2 = residual_unit(res_unit_1, 64, name='res_unit_2')
res_unit_2.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 3: Two stacked 64-filter 3 x 3 conv layers

In [10]:
res_unit_3 = residual_unit(res_unit_2, 64, name='res_unit_3')
res_unit_3.shape

TensorShape([Dimension(None), Dimension(56), Dimension(56), Dimension(64)])

### Residual Unit 4: Two stacked 128-filter 3 x 3 conv layers

In [11]:
res_unit_4 = residual_unit(res_unit_3, 128, name='res_unit_4')
res_unit_4.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 5: Two stacked 128-filter 3 x 3 conv layers

In [12]:
res_unit_5 = residual_unit(res_unit_4, 128, name='res_unit_5')
res_unit_5.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 6: Two stacked 128-filter 3 x 3 conv layers

In [13]:
res_unit_6 = residual_unit(res_unit_5, 128, name='res_unit_6')
res_unit_6.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 7: Two stacked 128-filter 3 x 3 conv layers

In [14]:
res_unit_7 = residual_unit(res_unit_6, 128, name='res_unit_7')
res_unit_7.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(128)])

### Residual Unit 8: Two stacked 256-filter 3 x 3 conv layers

In [15]:
res_unit_8 = residual_unit(res_unit_7, 256, name='res_unit_8')
res_unit_8.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### Residual 9: Two stacked 256-filter 3 x 3 conv layers

In [16]:
res_unit_9 = residual_unit(res_unit_8, 256, name='res_unit_9')
res_unit_9.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### Residual 10: Two stacked 256-filter 3 x 3 conv layers

In [17]:
res_unit_10 = residual_unit(res_unit_9, 256, name='res_unit_10')
res_unit_10.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### Residual 11: Two stacked 256-filter 3 x 3 conv layers

In [18]:
res_unit_11 = residual_unit(res_unit_10, 256, name='res_unit_11')
res_unit_11.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### Residual 12: Two stacked 256-filter 3 x 3 conv layers

In [19]:
res_unit_12 = residual_unit(res_unit_11, 256, name='res_unit_12')
res_unit_12.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### Residual 13: Two stacked 256-filter 3 x 3 conv layers

In [20]:
res_unit_13 = residual_unit(res_unit_12, 256, name='res_unit_13')
res_unit_13.shape

TensorShape([Dimension(None), Dimension(14), Dimension(14), Dimension(256)])

### Residual 14: Two stacked 512-filter 3 x 3 conv layers

In [21]:
res_unit_14 = residual_unit(res_unit_13, 512, name='res_unit_14')
res_unit_14.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(512)])

### Residual 15: Two stacked 512-filter 3 x 3 conv layers

In [22]:
res_unit_15 = residual_unit(res_unit_14, 512, name='res_unit_15')
res_unit_15.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(512)])

### Residual 16: Two stacked 512-filter 3 x 3 conv layers

In [23]:
res_unit_16 = residual_unit(res_unit_15, 512, name='res_unit_16')
res_unit_16.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(512)])

### AVG POOL: Global Average Pooling

In [24]:
avg_pool = global_average_pool(res_unit_16)
avg_pool.shape

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(512)])

### FC : Fully connected layer 

In [25]:
avg_pool_flattened = tf.reshape(avg_pool, [-1, 256])
fc = normal_full_layer(avg_pool_flattened, NUM_CLASSES)
fc.shape

TensorShape([Dimension(None), Dimension(10)])

### Softmax Classification Output

In [26]:
y_pred = tf.nn.softmax(fc)
y_pred.shape

TensorShape([Dimension(None), Dimension(10)])

### Loss Function

In [27]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



### Adam Optimizer

In [28]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001) # Adam optimizer
train = optimizer.minimize(cross_entropy)  # training operation

init = tf.global_variables_initializer() # global variables initializer